# Batch Processing

This is a script that helps you to batch process your code to solve for solution to different scenarios. It is composed of a configuration file (.yaml) and the codes below. Before you run this code, you should make sure the configuration file is set correctly. Here are the explanation for each of the parameters in the configuration file:
* input_path: the input directory of CommonRoad scenarios that you indend to solve.
* output_path: the output directory of the solution files.
* overwrite: the falg to determine whether to overwrite existing solution files.
* timeout: timeout time for your motion planner, unit in seconds
* trajectory_planner_path: input directory where the module containing the function to execute your motion planner is located
* trajectory_planner_module_name: name of the module taht contains the function to execute your motion planner
* trajectory_planner_function_name: name of the function that executes your motion planner
* default: the parameters specified under this will be applied to all scenarios. if you wish to specify a different paramter for specific scenarios, simply copy the section and replace 'default' with the id of your scenario.
* vehicle_model: model of the vehicle, its value could be PM, KS, ST and MB.
* vehicle_type type of the vehicle, its value could be FORD_ESCORT, BMW_320i and VW_VANAGON.
* cost_function: identifier of cost function. Please refer to [Cost Functions](https://gitlab.lrz.de/tum-cps/commonroad-cost-functions/blob/master/costFunctions_commonRoad.pdf) for more information.

### Helper functions

In [4]:
import os
import pathlib
import multiprocessing
import yaml
import sys
import warnings

from commonroad.common.file_reader import CommonRoadFileReader
from commonroad.common.solution_writer import CommonRoadSolutionWriter, VehicleModel, VehicleType, CostFunction

def parse_vehicle_model(model):
    if model == 'PM':
        cr_model = VehicleModel.PM
    elif model == 'ST':
        cr_model = VehicleModel.ST
    elif model == 'KS':
        cr_model = VehicleModel.KS
    elif model == 'MB':
        cr_model = VehicleModel.MB
    else:
        raise ValueError('Selected vehicle model is not valid: {}.'.format(model))
    return cr_model


def parse_vehicle_type(type):
    if type == 'FORD_ESCORT':
        cr_type = VehicleType.FORD_ESCORT
        cr_type_id = 1
    elif type == 'BMW_320i':
        cr_type = VehicleType.BMW_320i
        cr_type_id = 2
    elif type == 'VW_VANAGON':
        cr_type = VehicleType.VW_VANAGON
        cr_type_id = 3
    else:
        raise ValueError('Selected vehicle type is not valid: {}.'.format(type))
        
    return cr_type, cr_type_id


def parse_cost_function(cost):
    if cost == 'JB1':
        cr_cost = CostFunction.JB1
    elif cost == 'SA1':
        cr_cost = CostFunction.SA1
    elif cost == 'WX1':
        cr_cost = CostFunction.WX1
    elif cost == 'SM1':
        cr_cost = CostFunction.SM1
    elif cost == 'SM2':
        cr_cost = CostFunction.SM2
    elif cost == 'SM3':
        cr_cost = CostFunction.SM3
    else:
        raise ValueError('Selected cost function is not valid: {}.'.format(cost))
    return cr_cost


def call_trajectory_planner(queue, function, scenario, planning_problem_set, vehicle_type_id):
    queue.put(function(scenario, planning_problem_set, vehicle_type_id))

### Read configuration

In [5]:
# open config file
with open('batch_processing_config.yaml', 'r') as stream:
    try:
        settings = yaml.load(stream)
    except yaml.YAMLError as exc:
        print(exc)

# get planning wrapper function
sys.path.append(os.getcwd() + os.path.dirname(settings['trajectory_planner_path']))
module = __import__(settings['trajectory_planner_module_name'])
function = getattr(module, settings['trajectory_planner_function_name'])
time_timeout = settings['timeout']

/home/hans/anaconda3/envs/commonroad-py37/lib/python3.7/site-packages/ipykernel_launcher.py:4: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  after removing the cwd from sys.path.


### Start Processing

In [6]:
# iterate through scenarios
num_files = len(os.listdir(settings['input_path']))

print("Total number of files to be processed: {}".format(num_files))
print("Timeout setting: {} seconds\n".format(time_timeout))
    
count_processed = 0
for filename in os.listdir(settings['input_path']):
    count_processed += 1
    print("File No. {} / {}".format(count_processed, num_files))
    
    if not filename.endswith('.xml'):
        continue
        
    fullname = os.path.join(settings['input_path'], filename)

    print("Started processing scenario {}".format(filename))
    scenario, planning_problem_set = CommonRoadFileReader(fullname).open()

    # get settings for each scenario
    if scenario.benchmark_id in settings.keys():
        # specific
        vehicle_model = parse_vehicle_model(settings[scenario.benchmark_id]['vehicle_model'])
        vehicle_type,vehicle_type_id = parse_vehicle_type(settings[scenario.benchmark_id]['vehicle_type'])
        cost_function = parse_cost_function(settings[scenario.benchmark_id]['cost_function'])
    else:
        # default
        vehicle_model = parse_vehicle_model(settings['default']['vehicle_model'])
        vehicle_type, vehicle_type_id = parse_vehicle_type(settings['default']['vehicle_type'])
        cost_function = parse_cost_function(settings['default']['cost_function'])
        
    queue = multiprocessing.Queue()
    # create process, pass in required arguements
    p = multiprocessing.Process(target=call_trajectory_planner, name="trajectory_planner",
                                args=(queue, function, scenario, planning_problem_set, vehicle_type_id))
    # start planning
    p.start()
    
    # wait till process ends or skip if timed out
    p.join(timeout=time_timeout)

    if p.is_alive():
        print("===> Trajectory planner timeout.")
        p.terminate()
        p.join()
        solution_trajectories = {}
    else:
        print("Planning finished.")
        solution_trajectories = queue.get()

    # create path for solutions
    pathlib.Path(settings['output_path']).mkdir(parents=True, exist_ok=True)

    error = False
    cr_solution_writer = CommonRoadSolutionWriter(settings['output_path'], 
                                                  scenario.benchmark_id, 
                                                  scenario.dt,
                                                  vehicle_type, 
                                                  vehicle_model, 
                                                  cost_function)
    
    # inspect whether all planning problems are solved
    for planning_problem_id, planning_problem in planning_problem_set.planning_problem_dict.items():
        if planning_problem_id not in solution_trajectories.keys():
            print('Solution for planning problem with ID={} is not provided for scenario {}. Solution skipped.'.format(
                planning_problem_id, filename))
            error = True
            break
        else:
            cr_solution_writer.add_solution_trajectory(
                solution_trajectories[planning_problem_id], planning_problem_id)
    if not error:
        cr_solution_writer.write_to_file(overwrite=settings['overwrite'])

    print("=========================================================")

Total number of files to be processed: 294
Timeout setting: 120 seconds

File No. 1 / 294
Started processing scenario USA_US101-23_1_T-1.xml
Reading motion primitives...
Automata created.
Number of loaded primitives: 3066
Start search..

position= Rectangle: 
	 width: 1.6317 
	 length: 2.1212 
	 center: [ 173.0496 -137.6949] 
	 orientation: -0.72225 

orientation= Interval
start: -0.77835
end: -0.60381

velocity= Interval
start: 13.5903
end: 19.5903

time_step= Interval
start: 130
end: 140


Solving this scenario took 113.12 seconds
Planning finished.
File No. 2 / 294
Started processing scenario USA_US101-11_3_T-1.xml
Reading motion primitives...
Automata created.
Number of loaded primitives: 3066
Start search..

time_step= Interval
start: 72
end: 72


Solving this scenario took 7.82 seconds
Planning finished.
File No. 3 / 294
Started processing scenario USA_Lanker-1_2_T-1.xml
Reading motion primitives...
Automata created.
Number of loaded primitives: 3066
Start search..

position= Rec


Solving this scenario took 3.9 seconds
Planning finished.
File No. 17 / 294
Started processing scenario DEU_Gar-3_3_T-1.xml
Reading motion primitives...
Automata created.
Number of loaded primitives: 3066
Start search..

position= ShapeGroup: 
	 number of shapes: 1 

time_step= Interval
start: 204
end: 204


Solving this scenario took 0.28 seconds
Planning finished.
Solution for planning problem with ID=2 is not provided for scenario DEU_Gar-3_3_T-1.xml. Solution skipped.
File No. 18 / 294
Started processing scenario DEU_Muc-24_1_T-1.xml
Reading motion primitives...
Automata created.
Number of loaded primitives: 3066
Start search..

position= ShapeGroup: 
	 number of shapes: 1 

orientation= Interval
start: -3.3896
end: -3.1896

time_step= Interval
start: 194
end: 204


===> Trajectory planner timeout.
Solution for planning problem with ID=0 is not provided for scenario DEU_Muc-24_1_T-1.xml. Solution skipped.
File No. 19 / 294
Started processing scenario CHN_Sha-17_2_T-1.xml
Reading m

Reading motion primitives...
Automata created.
Number of loaded primitives: 3066
Start search..

position= Rectangle: 
	 width: 6.0 
	 length: 20.0 
	 center: [22.  18.5] 
	 orientation: 0.0 

orientation= Interval
start: -0.3927
end: 0.3927

velocity= Interval
start: 0.0
end: 15.0

time_step= Interval
start: 25
end: 31


Solving this scenario took 0.57 seconds
Planning finished.
Solution for planning problem with ID=9 is not provided for scenario RUS_Bicycle-3_2_T-1.xml. Solution skipped.
File No. 35 / 294
Started processing scenario CHN_Sha-3_1_T-1.xml
Reading motion primitives...
Automata created.
Number of loaded primitives: 3066
Start search..

position= Rectangle: 
	 width: 2.0 
	 length: 10.0 
	 center: [48.44368854 60.60724814] 
	 orientation: -1.722056957042279 

orientation= Interval
start: -1.8221
end: -1.6221

time_step= Interval
start: 155
end: 165


Solving this scenario took 104.13 seconds
Planning finished.
File No. 36 / 294
Started processing scenario USA_Lanker-1_9_T-

Automata created.
Number of loaded primitives: 3066
Start search..

position= ShapeGroup: 
	 number of shapes: 4 

time_step= Interval
start: 165
end: 165


===> Trajectory planner timeout.
Solution for planning problem with ID=1 is not provided for scenario CHN_Sha-11_4_T-1.xml. Solution skipped.
File No. 51 / 294
Started processing scenario USA_Peach-4_5_T-1.xml
Reading motion primitives...
Automata created.
Number of loaded primitives: 3066
Start search..

position= ShapeGroup: 
	 number of shapes: 4 

time_step= Interval
start: 70
end: 70


Solving this scenario took 37.28 seconds
Planning finished.
File No. 52 / 294
Started processing scenario USA_US101-14_1_T-1.xml
Reading motion primitives...
Automata created.
Number of loaded primitives: 3066
Start search..

position= Rectangle: 
	 width: 1.7543 
	 length: 2.2805 
	 center: [ 66.7131 -55.0908] 
	 orientation: -0.71949 

orientation= Interval
start: -0.79169
end: -0.61715

velocity= Interval
start: 13.0294
end: 19.0294

time_ste

Start search..

time_step= Interval
start: 19
end: 20


Solving this scenario took 1.23 seconds
Planning finished.
File No. 67 / 294
Started processing scenario CHN_Sha-10_1_T-1.xml
Reading motion primitives...
Automata created.
Number of loaded primitives: 3066
Start search..

position= Rectangle: 
	 width: 2.0 
	 length: 10.0 
	 center: [-10.73306477   9.88190503] 
	 orientation: 0.30205144352686375 

orientation= Interval
start: 0.2021
end: 0.4021

time_step= Interval
start: 155
end: 165


Solving this scenario took 0.4 seconds
Planning finished.
Solution for planning problem with ID=1 is not provided for scenario CHN_Sha-10_1_T-1.xml. Solution skipped.
File No. 68 / 294
Started processing scenario DEU_Muc-4_2_T-1.xml
Reading motion primitives...
Automata created.
Number of loaded primitives: 3066
Start search..

position= ShapeGroup: 
	 number of shapes: 1 

time_step= Interval
start: 50
end: 50


Solving this scenario took 2.65 seconds
Planning finished.
File No. 69 / 294
Started 

...


Solving this scenario took 55.66 seconds
Planning finished.
File No. 219 / 294
Started processing scenario CHN_Sha-2_5_T-1.xml
Reading motion primitives...
Automata created.
Number of loaded primitives: 3066
Start search..

position= ShapeGroup: 
	 number of shapes: 1 

time_step= Interval
start: 165
end: 165


Solving this scenario took 0.29 seconds
Planning finished.
Solution for planning problem with ID=2 is not provided for scenario CHN_Sha-2_5_T-1.xml. Solution skipped.
File No. 220 / 294
Started processing scenario USA_US101-12_1_T-1.xml
Reading motion primitives...
Automata created.
Number of loaded primitives: 3066
Start search..

position= Rectangle: 
	 width: 1.6371 
	 length: 2.1283 
	 center: [ 69.7154 -67.3121] 
	 orientation: -0.72962 

orientation= Interval
start: -0.80147
end: -0.62694

velocity= Interval
start: 12.2309
end: 18.2309

time_step= Interval
start: 70
end: 80


===> Trajectory planner timeout.
Solution for planning problem with ID=308 is not provided for sc


Solving this scenario took 44.39 seconds
Planning finished.
File No. 235 / 294
Started processing scenario CHN_Sha-7_2_T-1.xml
Reading motion primitives...
Automata created.
Number of loaded primitives: 3066
Start search..

position= ShapeGroup: 
	 number of shapes: 1 

time_step= Interval
start: 147
end: 147


Solving this scenario took 0.4 seconds
Planning finished.
Solution for planning problem with ID=1 is not provided for scenario CHN_Sha-7_2_T-1.xml. Solution skipped.
File No. 236 / 294
Started processing scenario DEU_Muc-1_1_T-1.xml
Reading motion primitives...
Automata created.
Number of loaded primitives: 3066
Start search..

position= Rectangle: 
	 width: 1.9906 
	 length: 2.9859 
	 center: [-12.0833   0.0519] 
	 orientation: -3.0002 

orientation= Interval
start: -0.3927
end: 0.3927

time_step= Interval
start: 0
end: 20


===> Trajectory planner timeout.
Solution for planning problem with ID=800 is not provided for scenario DEU_Muc-1_1_T-1.xml. Solution skipped.
File No. 23


===> Trajectory planner timeout.
Solution for planning problem with ID=75 is not provided for scenario USA_US101-14_2_T-1.xml. Solution skipped.
File No. 251 / 294
Started processing scenario CHN_Sha-6_3_T-1.xml
Reading motion primitives...
Automata created.
Number of loaded primitives: 3066
Start search..

position= Rectangle: 
	 width: 2.0 
	 length: 10.0 
	 center: [-3.44954445 52.82190363] 
	 orientation: -0.5092471455000879 

orientation= Interval
start: -0.6092
end: -0.4092

time_step= Interval
start: 155
end: 165


Solving this scenario took 39.34 seconds
Planning finished.
File No. 252 / 294
Started processing scenario DEU_A99-1_2_T-1.xml
Reading motion primitives...
Automata created.
Number of loaded primitives: 3066
Start search..

position= ShapeGroup: 
	 number of shapes: 1 

time_step= Interval
start: 30
end: 30


Solving this scenario took 7.19 seconds
Planning finished.
Solution for planning problem with ID=800 is not provided for scenario DEU_A99-1_2_T-1.xml. Solution 


===> Trajectory planner timeout.
Solution for planning problem with ID=2777 is not provided for scenario USA_Lanker-2_9_T-1.xml. Solution skipped.
File No. 268 / 294
Started processing scenario DEU_Muc-16_1_T-1.xml
Reading motion primitives...
Automata created.
Number of loaded primitives: 3066
Start search..

position= Rectangle: 
	 width: 2.0 
	 length: 10.0 
	 center: [-0.78594304 -8.86802472] 
	 orientation: -1.5239125700177165 

orientation= Interval
start: -1.6239
end: -1.4239

time_step= Interval
start: 117
end: 121


===> Trajectory planner timeout.
Solution for planning problem with ID=1 is not provided for scenario DEU_Muc-16_1_T-1.xml. Solution skipped.
File No. 269 / 294
Started processing scenario USA_Lanker-2_25_T-1.xml
Reading motion primitives...
Automata created.
Number of loaded primitives: 3066
Start search..

position= ShapeGroup: 
	 number of shapes: 3 

time_step= Interval
start: 100
end: 100


===> Trajectory planner timeout.
Solution for planning problem with I

Reading motion primitives...
Automata created.
Number of loaded primitives: 3066
Start search..

position= Rectangle: 
	 width: 1.6226 
	 length: 2.1094 
	 center: [ 137.8833 -111.6708] 
	 orientation: -0.7281 

orientation= Interval
start: -0.8059
end: -0.63137

velocity= Interval
start: 13.5354
end: 19.5354

time_step= Interval
start: 119
end: 129


===> Trajectory planner timeout.
Solution for planning problem with ID=63 is not provided for scenario USA_US101-24_1_T-1.xml. Solution skipped.
File No. 285 / 294
Started processing scenario USA_US101-27_3_T-1.xml
Reading motion primitives...
Automata created.
Number of loaded primitives: 3066
Start search..

time_step= Interval
start: 80
end: 80


Solving this scenario took 42.46 seconds
Planning finished.
File No. 286 / 294
Started processing scenario CHN_Sha-16_2_T-1.xml
Reading motion primitives...
Automata created.
Number of loaded primitives: 3066
Start search..

position= ShapeGroup: 
	 number of shapes: 3 

time_step= Interval
st